In [ ]:
import torch

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as pl
import scipy.signal as sg

pl.style.use('fivethirtyeight')
mpl.rcParams['axes.facecolor']='white'  
mpl.rcParams['figure.facecolor'] = '1'

from scipy.ndimage  import gaussian_filter1d

import json
import re
import os

#import copy

In [ ]:
from masking import *
from latencies import *
from excitation import *
from deconv import *
from ur import *
from tuning import *
from test import *
from ur import *

from data import CAPData


In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Simulate CAPs

In [ ]:
#masking degree function
#mdFunc=SigmoidMaskingDegreeFunction(30, 2*1/10., requires_grad=False)
mdFunc=WeibullCDF_IOFunc(0, 30, 7)
#latencies
lat0=Eggermont1976clickLatencies80dB #latencies

#Unitary response
#defined later
#ur0Wang=URWang1979m
#t=np.linspace(5e-4, 10e-3, num=500)
#u00=ur0Wang.u(t-2e-3) #shift
#sig=0.2e-3 #std of gaussian kernel in s
#sig2=sig/(t[1]-t[0])
#u0 = gaussian_filter1d(u00, sigma=sig2)
#u0/=np.amax(np.abs(u0))
#u0*=0.5*np.amax(np.abs(u00))

#tuning
BW10_0=500
BW10Func=constant_BW10(BW10_0, requires_grad=False)

plotMaskingDegreeFunc(mdFunc)
plotLatencies(lat0)

Example: two maskers

In [ ]:
#UR
#t2=np.linspace()
t=np.linspace(0.5, 7, 500)*1e-3


urWang=URWang1979shifted.u(t+0.5e-3)
urWangm = URWang1979m.u(t-1.5e-3)

dt=t[1]-t[0]
ur00Wang=gaussian_filter1d(urWang, 2e-4/dt)
ur0Wang=gaussian_filter1d(urWangm, 2e-4/dt)

pl.figure()
pl.title('Examples of unitary responses')
pl.plot(t*1e3, urWang, label='Wang')
pl.plot(t*1e3, urWangm, label='Wang modified')

pl.plot(t*1e3, ur00Wang, label='Wang + gaussian blur')
pl.plot(t*1e3, ur0Wang, label='Wang m + gaussian blur')
pl.legend()
pl.xlabel('t (ms)')

u0=ur0Wang

### Interactive plots

### Test data

In [ ]:
data_folder='./Data/Data-10-09/matFiles'

listFiles = os.listdir(data_folder)

listFilesMat=[]

for filename in listFiles:
    if re.match('.*.mat', filename):
        listFilesMat.append(filename)
listFilesMat.sort()

In [ ]:
#filter name
import re

filtered_filenames=[]
for filename in listFilesMat:
    includeFile=False
    for m in re.finditer(r'\d{4}', filename):
        if np.abs(int(m.group(0))-5000)<1200:
            includeFile=True
    m2 = re.match(r'.*hp', filename)
    if m2:
        includeFile=True
    m3= re.match(r'.*broadband', filename)
    if m3:
        includeFile=True
    m4=re.match(r'.*5k', filename)
    if m4:
        includeFile=True
    if includeFile:
        filtered_filenames.append(filename)

In [ ]:
capData=CAPData(data_folder, filtered_filenames, begin_ind=175, end_ind=912)

### Simulate CAP with existing maskers

In [ ]:
#masking amount

maskingConditions=capData.maskingConditions
I0=60
maskingConditions.set_amp0_dB(I0)


pl.figure(figsize=(8, 20))

plotMaskingAmountExcitations( BW10Func, maskingConditions, mdFunc, reg_ex='(.*notch5k)')
pl.show()

In [ ]:


#t=torch.tensor(capData.t)

t=torch.linspace(0.5, 12, 500)*1e-3

E=ExcitationPatterns.GammaExcitation(t, 1, 1.5, 1/(2e-3), 1e-3)
E.apply_Tukey_window(0.3)

pl.figure()


plotExcitationPatterns(E)
pl.show()

pl.figure(figsize=(10, 20))

E.set_masking_model(lat0, BW10Func, maskingConditions, mdFunc, filter_model='gaussian')

#maskingAmount, ex_patterns=E.get_tensors()
plotExcitationPatterns(E, plot_raw_excitation=False, reg_ex='(.*broadband)|(.*hp)')
pl.show()

In [ ]:

pl.figure(figsize=(12, 10))
plotSimulatedCAPs(E, u0,  shift=0.7e-3, ylim=[-6, 6], reg_ex='(.*hp)', max_plots=8)
pl.show()

### Estimation of parameters

Empirical estimation of IO curve

In [ ]:
reg_ex='(.*notch)|(.*broadband)'
u=u0
excs = E.get_tensor() 
for i, exc in zip(range(maskingConditions.n_conditions), excs):
    if not reg_ex is None:
        if not(re.match(reg_ex, maskingConditions.names[i])):
            continue 
    exc_np = exc.detach().numpy()
    CAP=np.convolve(exc_np, u, mode='full')
    maxValue=np.amax(CAP)
    minValue=np.amin(CAP)
    print(f'{maskingConditions.names[i]} amp : {maxValue-minValue}')

In [ ]:
attn_amp=[(20, 1.78), (31, 4.3), (55, 9.4), (37, 8), (40, 8.9), (28, 1.98), (34, 6.50), (45, 9.3), (50, 9.45), (22, 1.77), (25, 1.74)]

In [ ]:
attns, amps= zip(*attn_amp)
attns=np.array(attns)
amps=np.array(amps)
inds=np.argsort(attns)
amps=amps[inds]
attns=attns[inds]
Is=I0-attns
mask_amount=1-(amps-np.amin(amps))/(np.amax(amps)-np.amin(amps))


wbcdf=WeibullCDF_IOFunc()
wbcdf.fit_data(Is, mask_amount)

pl.title('IO curve')
pl.plot(Is, mask_amount, label='empirical')
I=torch.linspace(5, 50, steps=50)
pl.plot(I, wbcdf(I), label='fitted')
pl.plot(I, mdFunc(I), label='ground truth')
pl.xlabel('I (dB)')
pl.ylabel('Masking amount')
pl.legend()

Estimation of latencies

In [ ]:
f_lat=[(4500, 2.5), (4000, 2.75), (1000, 6), (9000, 1.8), (1500, 4.7), (5000, 2.4), (5500, 2.25), (2500, 3.4), (2000, 3.9)]

In [ ]:
fs, lats= zip(*f_lat)
fs=np.array(fs)
lats=np.array(lats)*1e-3
inds=np.argsort(lats)
lats=lats[inds]
fs=fs[inds]

t=torch.linspace(1.4, 12, 500)*1e-3

lat=PowerLawLatencies()
lat.fit_data(lats, fs)

pl.title('Latencies')
pl.plot(lats*1e3, fs, label='empirical')
pl.plot(t*1e3, lat0.f_from_t(t), label='ground truth')
pl.plot(t*1e3, lat.f_from_t(t), label='fitted')
pl.xlabel('t (ms)')
pl.ylabel('f')
pl.legend()

In [ ]:
#get CAP signals
sig_CAP_noise=0.3

u=u0
excs = E.get_tensor()
CAP_arr_list=[]

shift=0

t=E.t.numpy()
ind_time=np.sum(t<(t[0]+shift))
            
for i, exc in zip(range(maskingConditions.n_conditions), excs):
    exc_np = exc.detach().numpy()
    CAP=np.convolve(exc_np, u, mode='full')
    if i==0:
        ind_time=min(ind_time, len(CAP)-len(E.t))
    CAP=CAP[ind_time:ind_time+len(E.t)]
    CAP_arr_list.append(CAP)
    
CAPs0=np.stack(CAP_arr_list, axis=0)

CAPs0_noisy=CAPs0+sig_CAP_noise*np.random.randn(*np.shape(CAPs0))

pl.figure(figsize=(12, 3))
plotSimulatedCAPs(E, CAParray=CAPs0_noisy ,shift=0.7e-3, ylim=[-6, 6], max_plots=2, title='Noisy simulated CAPs') #reg_ex='(.*hp)', 
pl.show()

In [ ]:
#make a guess on raw excitation pattern
E1=ExcitationPatterns.GammaExcitation(t, 1, 3, 1/(1e-3), 0.4e-3)

plotExcitationPatterns(E1)

#### Non-blind deconvolution

In [ ]:
CAPs_f=np.zeros_like(CAPs0_noisy)
window=sg.tukey(np.shape(CAPs0_noisy)[1], alpha=0.2)
for i in range(maskingConditions.n_conditions):
    #window signal (tuckey window)
    CAP_w=window*CAPs0_noisy[i]
    CAPs_f[i]=sg.wiener(CAP_w)
    #low pass (gaussian kernel to simplify for now)
    sig_f=0.2e-3 #in s
    dt=(t[1]-t[0])
    sig_f=sig_f/dt
    CAPs_f[i]=gaussian_filter1d(CAPs_f[i], sigma=sig_f)
    
pl.figure(figsize=(12, 3))
axs= plotSimulatedCAPs(E, CAParray=CAPs0_noisy,shift=0.7e-3, ylim=[-6, 6], max_plots=2) #reg_ex='(.*hp)', 
plotSimulatedCAPs(E, CAParray=CAPs_f,shift=0.7e-3, ylim=[-6, 6], max_plots=2, axlist=axs, title='Noisy simulated CAPs + filtered CAPs')
pl.show()

In [ ]:
#deconvs

#given u
u_fft=np.fft.rfft(u0)
EP_deconv=np.zeros_like(CAPs_f)
for i in range(maskingConditions.n_conditions):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=CAP_fft/u_fft
    EP_deconv[i]=np.fft.irfft(EP_fft)
    
#given EPs
EPs0=E.get_tensor().detach().numpy()
u_deconv=np.zeros_like(CAPs_f)
for i in range(maskingConditions.n_conditions):
    CAP_fft=np.fft.rfft(CAPs_f[i])
    EP_fft=np.fft.rfft(EPs0[i])
    u_fft=CAP_fft/EP_fft
    u_deconv[i]=np.fft.irfft(u_fft)
    
EP_deconv0=np.copy(EP_deconv)
u_deconv0=np.copy(u_deconv)

In [ ]:
def plotMatrices(X, X_ref, title='', max_plots=6):
    pl.figure(figsize=(12, 5))
    pl.suptitle(title)
    m=maskingConditions.n_conditions
    m=min(m, max_plots)
    nb_col=(m+1)//2 if m<=12 else (m+2)//3
    nb_row=(m+nb_col-1)//nb_col
    for i in range(m):
        ind=i+1
        pl.subplot(nb_row, nb_col, ind)
        pl.plot(t*1e3, X_ref[i])
        pl.plot(t*1e3, X[i])
        pl.xlabel('t (ms)')
        pl.gca().get_yaxis().set_visible(False)
    pl.tight_layout()
    pl.show()
    

In [ ]:
EPs0=E.get_tensor().detach().numpy()
plotMatrices(EP_deconv0, EPs0, title='Test deconv of EPs (if u is known), before proj')

In [ ]:
u0_mat=np.tile(u0, (maskingConditions.n_conditions, 1))
plotMatrices(u_deconv0, u0_mat, title='Test deconv of u (if EPs are known), before proj')

Test projection of estimated excitation patterns with and without intercept (nonmaskable part)  (NB: it'd be possible to add a regularity prior)

In [ ]:
# with gradient descent (/Newton algorithm) + projs
#given u
u_fft=np.fft.rfft(u0)

EP1=ExcitationPatterns.copyRaw(E1)

EP1.set_masking_model(lat0, BW10Func, maskingConditions, mdFunc, filter_model='gaussian')  #same params as generation of CAPs

#EP1.set_masking_model(lat0, BW10Func, maskingConditions, wbcdf, filter_model='gaussian')  #estimated params for IO curve



nb_steps=6
alpha=np.linspace(0.6, 0.1, nb_steps)
u_fft=np.fft.rfft(u0)
CAPs_fft=np.fft.rfft(CAPs_f, axis=1)
proj_EPs=EP1.get_projector(intercept=True)
proj_EPs_wo_intercept=EP1.get_projector(intercept=False)

#init:deconv
#EP_deconv=np.copy(EP_deconv0)
#EP_deconv_wo_i=np.copy(EP_deconv0)

#init EP1
EP_deconv=EP1.get_tensor().numpy()
EP_deconv_wo_i=EP1.get_tensor().numpy()
EP1_wo_i_E0=EP1.E0_maskable

for i in range(1, nb_steps+1):
    if i%2==1:
        pass
        #plotMatrices(EP_deconv, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent step {i}')
        '''
        pl.figure()
        pl.title(f'Step {i-1}')
        pl.plot(t, E.E0_maskable, label='E0 ground truth')
        pl.plot(t, EP1_wo_i_E0, label='E0 wo int. (estimated)')
        pl.legend()
        pl.show()
        '''
    dEP=deconv_newton_step(EP_deconv, u_fft, CAPs_fft)
    dEP_wo_i=deconv_newton_step(EP_deconv_wo_i, u_fft, CAPs_fft)
    EP_deconv-=alpha[i-1]*dEP
    EP_deconv_wo_i-=alpha[i-1]*dEP_wo_i
    #proj
    #from doc
    #intercept: if True, the function returned will return a nested tuple
    #( (E0_maskable, E0_nonmaskable), E0_nonmaskable+ (1-M) E0_maskable 
    # else it will return (E0, (1-M) E0)
    
    tup, EP_deconv=proj_EPs(EP_deconv)
    EP1_wo_i_E0, EP_deconv_wo_i =proj_EPs_wo_intercept(EP_deconv_wo_i)

    EP1_E0_m, EP1_E0_nm = tup 
plotMatrices(EP_deconv_wo_i, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent + proj (wo intercept)')

plotMatrices(EP_deconv, EPs0, title=f'Test deconv of EPs (u is known) w/ grad. descent + proj, (w/ intercept)')



pl.figure()
pl.plot(t, E.E0_maskable, label='E0 ground truth')
pl.plot(t, EP1_wo_i_E0, label='E0 wo int. (estimated)')

pl.plot(t, EP1_E0_m, label='E0 w int., maskable (estimated)')
pl.plot(t, EP1_E0_nm, label='E0 w int., non maskable (estimated)')
pl.legend()
pl.show()



In [ ]:
#given EPs
nb_steps=15
alpha=np.linspace(0.6, 0.1, nb_steps)
#other EP
#capSimulator1 = ConvolutionCAPSimulatorSingleFilterModel(lat0, gf0, EP1, md, mcs, ur=u0)
#EPs1=capSimulator1.getEPs()
#EPs_fft=np.fft.rfft(EPs1, axis=1)
EPs_fft=np.fft.rfft(EPs0, axis=1)
#u0_mat=np.tile(u0, (maskingConditions.n_conditions, 1))
u1_mat=np.zeros_like(CAPs_f)
filter_mat = t>6e-3
filter_mat=np.tile(filter_mat, (maskingConditions.n_conditions, 1))
for i in range(1, nb_steps+1):
    du=deconv_newton_step(u1_mat, EPs_fft, CAPs_fft)
    u1_mat-=alpha[i-1]*du
    #proj 1 
    u1_mat[filter_mat]=np.zeros_like(u1_mat[filter_mat])
    #proj 2
    u1_mat_mean=np.mean(u1_mat, axis=0)[None, :]
    u1_mat=np.repeat(u1_mat_mean, maskingConditions.n_conditions, axis=0)
    if i==nb_steps:
        pl.figure()
        pl.title(f'Step {i} (deconv + proj, EPs are known)')
        pl.plot(t, u0, label='u0 (truth)')
        pl.plot(t, u1_mat[0], label='u0 (estimated)')
        pl.legend()
        pl.show()